# 에필로그

오프닝: 지금까지 오픈 소스로 개발 중인 애저 쿠버네티스 엔진을 이용한 커스텀 쿠버네티스 클러스터를 구축하는 방법을 살펴보셨습니다.

Q: 지금까지 살펴본 내용들은 각각 사용할 수도 있지만 함께 적용할 수도 있나요?

A: 네. AKS 엔진을 사용하면 아직 정식으로 출시되지 않은 기능을 이용할 수도 있지만, 기존에 나와있던 기능들을 한 데 묶어 복잡하고 정교한 쿠버네티스 클러스터를 쉽게 만들 수도 있습니다.

Q: 이번 시리즈에서 소개하지 못한 기능들도 있을 것 같은데, 무엇이 더 있을까요?

A: 인텔에서는 SGX라는 보안 확장 기능을 스카이레이크 아키텍처를 탑재한 프로세서부터 본격적으로 제공하고 있는데, 이 기능을 사용하여 CPU 레벨에서 데이터 보안을 강화하는 기능을 제공합니다.

이 기능을 사용하면 민감한 개인 정보를 다루는 다양한 곳에서 훨씬 강력한 보안을 유지하면서도, 계속해서 쿠버네티스의 편리함을 누릴 수 있습니다.

[https://github.com/Azure/aks-engine/blob/master/docs/topics/sgx.md](https://github.com/Azure/aks-engine/blob/master/docs/topics/sgx.md)

A: 추후 IPv4 외에도 IPv6 주소를 사용하는 때를 대비하여, 양쪽의 주소 체계를 모두 활용할 수 있는 듀얼 스택 배포도 AKS 엔진을 이용하여 쉽게 수행할 수 있습니다.

[https://github.com/Azure/aks-engine/tree/master/examples/dualstack](https://github.com/Azure/aks-engine/tree/master/examples/dualstack)

또한 여러 가지 애드온을 이용하여 쿠버네티스 클러스터의 기능을 더욱 풍부하게 만들고, 안정성을 더욱 강화할 수도 있습니다.

[https://github.com/Azure/aks-engine/tree/master/examples/addons](https://github.com/Azure/aks-engine/tree/master/examples/addons)

A: 만약 근무하시는 회사에서 애저 스택을 도입할 계획이시라면, 이번 시리즈에서 설명한 내용을 그대로 여러분이 구축할 프라이빗 데이터센터에 적용할 수 있습니다.

애저 스택 허브 또는 최근에 프리뷰 버전으로 출시된 애저 스택 HCI 중 적절한 솔루션을 도입할 수 있습니다.

무엇보다도 좋은 것은, 윈도우 컨테이너를 통해 오래된 윈도우용 레거시 백오피스 애플리케이션을 현대화할 수 있다는 것입니다.

[https://github.com/Azure/aks-engine/blob/master/examples/azure-stack/kubernetes-azurestack.json](https://github.com/Azure/aks-engine/blob/master/examples/azure-stack/kubernetes-azurestack.json)

Q: 마지막으로 하나 더 궁금한게 있습니다. 제가 알기로 쿠버네티스 클러스터는 어느 순간 관리해야 할 요소들이 매우 많아지던데요, 이럴 때 적용할 수 있는 묘안이 있을까요?

A: 이번 시리즈에서 소개한 AKS 엔진 기반의 쿠버네티스 클러스터 뿐 아니라, 아마 여러분께서는 시중 곳곳에서 쿠버네티스 클러스터를 쉽게 구축할 수 있다고 이야기하는 제품이나 솔루션들을 많이 보셨을겁니다.

A: 만약 애저를 기반으로 관리 거점을 잡고 계시다면, 애저 아크에 대해서도 살펴보실 것을 권해드립니다.

애저 아크를 사용하면 서버, 데이터베이스, 그리고 쿠버네티스 클러스터를 애저의 관리 모델 아래에 놓고 일관성있게 관리하고 모니터링할 수 있습니다. 자세한 내용은 화면에 보여지는 링크를 참고해주세요.

https://bit.ly/rkttu-azure-arc

클로징: 이 동영상을 보고 애저를 써봐야겠다고 생각하셨나요? 애저 계정은 누구나 무료로 만들 수 있습니다. 지금 바로 시작해보세요. (가능하다면 할인/무료 쿠폰을 포함한 가입 링크 제공) 다음 에피소드에서는 클라우드 셸을 사용할 수 있는 다양한 방법을 알아보겠습니다.